In [1]:
import dask.dataframe as dask_pd
import dask_geopandas as dask_gpd
import geopandas as gpd
import pandas as pd
from dask.distributed import Client
import time

In [2]:
pd.set_option('displayzz.width', 500)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [3]:
client = Client(n_workers=4, threads_per_worker=2, processes=True, env={"MALLOC_TRIM_THRESHOLD_":0})

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38001 instead
  warnings.warn(


In [4]:
dask_df_mdt =  dask_pd.read_csv('sample-folder/mdt_result.csv', assume_missing=True)
grid  =  dask_pd.read_csv('csv_polygon/inbuilding.csv', assume_missing=True)
cell_contributor = dask_pd.read_csv('csv_polygon/Summary-RSRP-CellContributor-Inbuilding-1.csv')

# dask_df_mdt = dask_df_mdt[(dask_df_mdt['absolute_rf_channel_number'] > 1200) & (dask_df_mdt['absolute_rf_channel_number'] < 1949)]

In [5]:
dask_df_mdt['rsrp'] = 'rsrp'

dask_df_mdt['pointer'] = "POINT (" + dask_df_mdt['longitude'].astype(str) + " " + dask_df_mdt['latitude'].astype(str) + ")"

dask_gdf_mdt = dask_gpd.from_dask_dataframe(dask_df_mdt, geometry=dask_df_mdt["pointer"].map_partitions(gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)
#dask_gdf_mdt = dask_gdf_mdt.set_crs(4326)

dask_gdf_grid = dask_gpd.from_dask_dataframe(grid, geometry=grid['WKT'].map_partitions(gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)
#dask_gdf_grid = dask_gdf_grid.set_crs(4326)

In [6]:
dask_gdf_mdt = dask_gdf_mdt.sjoin(dask_gdf_grid, how='inner', predicate='intersects')
dask_gdf_mdt = dask_gdf_mdt.drop(columns=['pointer'])

In [7]:
dask_gdf_mdt['Concatenate'] = dask_gdf_mdt['polygon_name'].astype(str) + "|" + dask_gdf_mdt['site'].astype(str) + "|" + dask_gdf_mdt['enodebid'].astype(int).astype(str) + "|" + dask_gdf_mdt['ci'].astype(int).astype(str)

In [8]:
dask_gdf_mdt['Concatenate'] = dask_gdf_mdt['Concatenate'].astype(str)

cell_contributor['Concatenate'] = cell_contributor['Concatenate'].astype(str)
result = dask_gdf_mdt.merge(cell_contributor, how='inner', on='Concatenate')

In [9]:
dask_gdf_mdt.head(10)

,date,hour,site,enodebid,ci,mmes1apid,longitude,latitude,altitude,rsrp_serving,rsrq_serving,earfcn_neigh,pci_1,rsrp_1,rsrq_1,pci_2,rsrp_2,rsrq_2,pci_3,rsrp_3,rsrq_3,pci_4,rsrp_4,rsrq_4,support_L900_FDD,support_L1800_FDD,support_L2600_TDD,CA_capable,CA_3CC_capable,MIMO4x4,ENDC,NR_SA,rsrp,geometry,index_right,polygon_name,WKT,Concatenate
1136,20230601.0,0.0,4445166E-LTE-HALIM-GOLF-RELOCATION,440290.0,3.0,296750057.0,106.886149,-6.265640,0.0,-99.0,-19.0,3774.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rsrp,POINT (106.88615 -6.26564),792,799_Bandarahalimpk,"POLYGON ((106.8865025 -6.265421, 106.88649817347691 -6.265509068400586, 106.88648523557444 -6.265596288654331, 106.88646381089166 -6.265681820782513, 106.88643410575996 -6.26576484106398, 106.88639640625601 -6.265844549968038, 106.88635107544665 -6.265920179854368, 106.88629854989235 -6.265991002365821, 106.8862393354429 -6.266056335442896, 106.88617400236582 -6.266115549892346, 106.88610317985437 -6.266168075446654, 106.88602754996803 -6.266213406256017, 106.88594784106398 -6.26625110575996...",799_Bandarahalimpk|4445166E-LTE-HALIM-GOLF-RELOCATION|440290|3
321177,20230601.0,0.0,444PC050E-LTE-BANDARA-HALIM,440532.0,7.0,762844789.0,106.885720,-6.264964,44.0,-78.0,-4.5,325.0,408.0,-83.0,-12.5,105.0,-101.0,-19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rsrp,POINT (106.88572 -6.26496),792,799_Bandarahalimpk,"POLYGON ((106.8865025 -6.265421, 106.88649817347691 -6.265509068400586, 106.88648523557444 -6.265596288654331, 106.88646381089166 -6.265681820782513, 106.88643410575996 -6.26576484106398, 106.88639640625601 -6.265844549968038, 106.88635107544665 -6.265920179854368, 106.88629854989235 -6.265991002365821, 106.8862393354429 -6.266056335442896, 106.88617400236582 -6.266115549892346, 106.88610317985437 -6.266168075446654, 106.88602754996803 -6.266213406256017, 106.88594784106398 -6.26625110575996...",799_Bandarahalimpk|444PC050E-LTE-BANDARA-HALIM|440532|7
321178,20230601.0,0.0,444PC050E-LTE-BANDARA-HALIM,440532.0,8.0,762844789.0,106.885270,-6.265254,36.0,-89.0,-7.5,325.0,416.0,-104.0,-19.5,105.0,-105.0,-19.5,374.0,-106.0,-19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rsrp,POINT (106.88527 -6.26525),792,799_Bandarahalimpk,"POLYGON ((106.8865025 -6.265421, 106.88649817347691 -6.265509068400586, 106.88648523557444 -6.265596288654331, 106.88646381089166 -6.265681820782513, 106.88643410575996 -6.26576484106398, 106.88639640625601 -6.265844549968038, 106.88635107544665 -6.265920179854368, 106.88629854989235 -6.265991002365821, 106.8862393354429 -6.266056335442896, 106.88617400236582 -6.266115549892346, 106.88610317985437 -6.266168075446654, 106.88602754996803 -6.266213406256017, 106.88594784106398 -6.26625110575996...",799_Bandarahalimpk|444PC050E-LTE-BANDARA-HALIM|440532|8
321179,20230601.0,0.0,444PC050E-LTE-BANDARA-HALIM,440532.0,5.0,149426645.0,106.885527,-6.265597,40.0,-78.0,-5.5,1313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rsrp,POINT (106.88553 -6.26560),792,799_Bandarahalimpk,"POLYGON ((106.8865025 -6.265421, 106.88649817347691 -6.265509068400586, 106.88648523557444 -6.265596288654331, 106.88646381089166 -6.265681820782513, 106.88643410575996 -6.26576484106398, 106.88639640625601 -6.265844549968038, 106.88635107544665 -6.265920179854368, 106.88629854989235 -6.265991002365821, 106.8862393354429 -6.266056335442896, 106.88617400236582 -6.266115549892346, 106.88610317985437 -6.266168075446654, 106.88602754996803 -6.266213406256017, 106.88594784106398 -6.26625110575996...",799_Bandarahalimpk|444PC050E-LTE-BANDARA-HALIM|440532|5
321180,20230601.0,0.0,444PC050E-LTE-BANDARA-HALIM,440532.0,8.0,762844789.0,106.884819,-6.265318,46.0,-95.0,-9.5,325.0,416.0,-103.0,-18.0,105.0,-103.0,-17.0,374.0,-109.0,-19.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rsrp,POINT (106.88482 -6.26532),792,799_Bandarahalimpk,"POLYGON ((106.8865025 -6.265421, 106.88649817347691 -6.265509068400586, 106.88648523557444 -6.2

In [10]:
cell_contributor.head(10)

,Concatenate,rsrp-pop,Sum of rsrp-pop,RSRP-AVG,Review (Detail Reason)
0,986_Plzherodpk|4462552E_CO|460805|4,115943,33%,-96.453704,"NOT GO -> Small Building, covered by Surrounding"
1,508_Rsgrahakedoya-IBS|MC4424192E_CO|420291|4,102896,46%,-96.577626,NOT GO -> Have Pico
2,330_Htltunepsbaru-MAC|4415460E-LTE-Pasar-Baru-Samanhudi|410925|3,74603,35%,-100.760504,NOT GO -> Covered by Surrounding
3,788_Rusunamikalibata1|441PXH259E_CO|430181|6,70483,10%,-97.450512,NOT GO -> Have Pico
4,802_Snipertaminisquare|4441021E_C2|440679|3,69583,10%,-95.909091,NOT GO -> Have Pico
5,1027_PASAR CENGKARENG|4423438E|420150|18,69091,33%,-95.429752,OK -> GO
6,802_Snipertaminisquare|4441021E_CO|440517|5,62582,9%,-95.307018,NOT GO -> Have Pico
7,162_Univtarumanegara2Adicipta|4425492E-LTE-GROGOL-FLYOVER|420358|18,62126,34%,-97.468354,NOT GO -> Have Pico
8,746_Melawaiplaza|MC4431405E_CO|430150|4,60580,41%,-96.357955,NOT GO -> Have Pico
9,508_Rsgrahakedoya-IBS|MC4424192E_CO|420291|7,60009,27%,-97.734584,NOT GO -> Have Pico


In [ ]:
dask_gdf_mdt.to_csv('result/test.csv')